### 스마일게이트 UnsmileData Pretrained model 사용해보기

참고자료: https://github.com/smilegate-ai/korean_unsmile_dataset

In [3]:
from datasets import load_dataset
datasets = load_dataset('smilegate-ai/kor_unsmile')

Downloading: 100%|██████████| 1.58k/1.58k [00:00<00:00, 790kB/s]
Using custom data configuration smilegate-ai--kor_unsmile-1dba960877497f9f


Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 499.71it/s]


Dataset parquet downloaded and prepared to C:\Users\Erin\.cache\huggingface\datasets\parquet\smilegate-ai--kor_unsmile-1dba960877497f9f\0.0.0\0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 111.10it/s]

DatasetDict({
    train: Dataset({
        features: ['문장', '여성/가족', '남성', '성소수자', '인종/국적', '연령', '지역', '종교', '기타 혐오', '악플/욕설', 'clean', '개인지칭', 'labels'],
        num_rows: 15005
    })
    valid: Dataset({
        features: ['문장', '여성/가족', '남성', '성소수자', '인종/국적', '연령', '지역', '종교', '기타 혐오', '악플/욕설', 'clean', '개인지칭', 'labels'],
        num_rows: 3737
    })
})


In [4]:
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['문장', '여성/가족', '남성', '성소수자', '인종/국적', '연령', '지역', '종교', '기타 혐오', '악플/욕설', 'clean', '개인지칭', 'labels'],
        num_rows: 15005
    })
    valid: Dataset({
        features: ['문장', '여성/가족', '남성', '성소수자', '인종/국적', '연령', '지역', '종교', '기타 혐오', '악플/욕설', 'clean', '개인지칭', 'labels'],
        num_rows: 3737
    })
})


실제 훈련된 모델은 '개인지칭'과 'label'은 빠져있음.
'개인지칭'의 경우 추가 정보(개인을 지칠할 경우 모욕적일 수도 있어서..?)

In [5]:
from transformers import TextClassificationPipeline, BertForSequenceClassification, AutoTokenizer
model_name = 'smilegate-ai/kor_unsmile'
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
pipe = TextClassificationPipeline(
        model = model,
        tokenizer = tokenizer,
        device = 0,   # cpu: -1, gpu: gpu number
        return_all_scores = True,
        function_to_apply = 'sigmoid'
    )


Downloading: 100%|██████████| 1.52k/1.52k [00:00<00:00, 1.50MB/s]
Downloading: 100%|██████████| 436M/436M [00:39<00:00, 11.2MB/s] 
Downloading: 100%|██████████| 370/370 [00:00<00:00, 185kB/s]
Downloading: 100%|██████████| 250k/250k [00:00<00:00, 276kB/s] 
Downloading: 100%|██████████| 721k/721k [00:01<00:00, 493kB/s]  
Downloading: 100%|██████████| 112/112 [00:00<00:00, 37.3kB/s]


{'label': '여성/가족', 'score': 0.8253053426742554}
{'label': '남성', 'score': 0.03972514718770981}
{'label': '성소수자', 'score': 0.01214433740824461}
{'label': '인종/국적', 'score': 0.023181889206171036}
{'label': '연령', 'score': 0.010315303690731525}
{'label': '지역', 'score': 0.01845489628612995}
{'label': '종교', 'score': 0.011270025745034218}
{'label': '기타 혐오', 'score': 0.020734043791890144}
{'label': '악플/욕설', 'score': 0.057331427931785583}
{'label': 'clean', 'score': 0.1401052176952362}


In [188]:
# 예시 문장
for result in pipe("ㅁㅊ 여러분 제발 뭐로든 신고하고 ㅇㅅㄹ 당선되면 나 이민가")[0]:
    print(result)

{'label': '여성/가족', 'score': 0.015001693740487099}
{'label': '남성', 'score': 0.00991620309650898}
{'label': '성소수자', 'score': 0.009861568920314312}
{'label': '인종/국적', 'score': 0.01520923338830471}
{'label': '연령', 'score': 0.008798819966614246}
{'label': '지역', 'score': 0.018083980306982994}
{'label': '종교', 'score': 0.012537132017314434}
{'label': '기타 혐오', 'score': 0.024936573579907417}
{'label': '악플/욕설', 'score': 0.8759555816650391}
{'label': 'clean', 'score': 0.09106995910406113}


갖고 있는 데이터로 테스트

In [20]:
import numpy as np
import pandas as pd

In [177]:
df3 = pd.read_csv('./소신투표/소신투표_result.csv', encoding='utf-8')

In [178]:
result1 = []
for t in df3['text'].astype(str):
    result1.extend(pipe(t))

In [189]:
# 결과 예시
result1[0]

[{'label': '여성/가족', 'score': 0.01607031561434269},
 {'label': '남성', 'score': 0.003696973202750087},
 {'label': '성소수자', 'score': 0.006438276264816523},
 {'label': '인종/국적', 'score': 0.020544659346342087},
 {'label': '연령', 'score': 0.007808662950992584},
 {'label': '지역', 'score': 0.030384093523025513},
 {'label': '종교', 'score': 0.010064619593322277},
 {'label': '기타 혐오', 'score': 0.03580975532531738},
 {'label': '악플/욕설', 'score': 0.21212667226791382},
 {'label': 'clean', 'score': 0.3380521237850189}]

In [160]:
label =  ['여성/가족', '남성', '성소수자', '인종/국적', '연령', '지역', '종교', '기타 혐오', '악플/욕설', 'clean']

In [180]:
score = [[item['score'] for item in i] for i in result1]

In [181]:
score = np.round(score,2)

In [186]:
df3.to_csv("./소신투표/소신투표_unsmile.csv", encoding='utf-8-sig',sep=',', index=False)